In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
ms_files =[]
sizes = []
for file in glob.glob('example-01/2020_05_11RG_SA001_RPT001/*.mzXML'):
    ms_files.append(file)
    sizes.append(os.path.getsize(file))

In [3]:
ms_files = pd.DataFrame({'file_name': ms_files, 'FILE_SIZE':sizes})
ms_files.head(2)

,file_name,FILE_SIZE
0,example-01/2020_05_11RG_SA001_RPT001/2020_05_1...,17618833
1,example-01/2020_05_11RG_SA001_RPT001/2020_05_1...,22298335


In [4]:
ms_files['plate'] = ms_files.file_name.str.split('LSARP_').apply(lambda x: x[1]).str.split('_').apply(lambda x: x[0])
ms_files.head(2)

,file_name,FILE_SIZE,plate
0,example-01/2020_05_11RG_SA001_RPT001/2020_05_1...,17618833,SA001
1,example-01/2020_05_11RG_SA001_RPT001/2020_05_1...,22298335,SA001


In [5]:
info_df = pd.read_csv('LSARP-Full-May2020-Worklist.csv',skiprows=1)
info_df.head(2)

,Sample Type,File Name,Sample ID,Path,Instrument Method,Process Method,Calibration File,Position,Inj Vol,Level,...,Sample Vol,ISTD Amt,Dil Factor,L1 Study,L2 Client,L3 Laboratory,L4 Company,L5 Phone,Comment,Sample Name
0,Unknown,2020_04_20RG_HILICPos15S_Col001_LSARP_SA008_Bl...,NaN,F:\QEHF\LRG\LSARP\2020_04_20RG_SA008,F:\QEHF\Methods\LSARP\2019_05_07RG_MultiHILICP...,NaN,NaN,G:C11,2,NaN,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Unknown,2020_04_20RG_HILICNeg15S_Col002_LSARP_SA008_Bl...,NaN,F:\QEHF\LRG\LSARP\2020_04_20RG_SA008,F:\QEHF\Methods\LSARP\2019_05_07RG_MultiHILICN...,NaN,NaN,G:C11,2,NaN,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
lsin = []
for i in range(len(info_df)):
    lsin.append('LSARP' in info_df['File Name'][i])
    
info_df['lsin'] = lsin
info_df = info_df[info_df.lsin == True]
info_df = info_df.drop(['lsin'],axis =1)

In [7]:
info_df['plate'] = info_df['File Name'].str.split('LSARP_').apply(lambda x: x[1]).str.split('_').apply(lambda x: x[0])

In [8]:
plin = []
for i in range(len(info_df)):
    plin.append(info_df.plate[i] in np.unique(ms_files.plate))
    
info_df['plin'] = plin
info_df = info_df[info_df.plin == True]
info_df = info_df.drop(['plin'],axis =1)

In [9]:
info_df = info_df.reset_index()
info_df = info_df.drop(['index'],axis = 1)

In [10]:
ms_files.file_name = ms_files.file_name.str.split('/').apply(lambda x: x[-1][:-6])
ms_files.head(2)

,file_name,FILE_SIZE,plate
0,2020_05_11RG_HILICPos15S_Col001_LSARP_SA001_RP...,17618833,SA001
1,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,22298335,SA001


In [11]:
out_df = info_df.copy()
out_df = out_df.drop(['Sample Type','Sample ID','Path','Level','Instrument Method',
                      'Calibration File','Sample Wt','Sample Vol','L1 Study','L2 Client',
                      'L3 Laboratory','L4 Company','L5 Phone','Comment','Process Method',
                      'Inj Vol','ISTD Amt','Dil Factor','Sample Name'],axis = 1)
out_df.head(2)

,File Name,Position,plate
0,2020_05_11RG_HILICPos15S_Col001_LSARP_SA001_RP...,G:C11,SA001
1,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:C11,SA001


In [12]:
ms_files = ms_files.sort_values(by = ['plate','file_name'], ascending = True)
ms_files = ms_files.reset_index()
ms_files = ms_files.drop(['index'], axis = 1)
ms_files.head(2)

,file_name,FILE_SIZE,plate
0,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,22479052,SA001
1,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,22452139,SA001


In [13]:
out_df = out_df.sort_values(by = ['plate','File Name'], ascending = True)
out_df = out_df.reset_index()
out_df = out_df.drop(['index'], axis =1 )
out_df.head()

,File Name,Position,plate
0,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A2,SA001
1,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A3,SA001
2,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A4,SA001
3,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A5,SA001
4,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A6,SA001


In [14]:
out_df['File Name'][0]

'2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RPT001_BI_16_0013'

In [15]:
ms_files.file_name[0]

'2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RPT001_BI_16_0013'

In [16]:
np.unique(out_df['File Name'] == ms_files.file_name)

array([ True])

In [17]:
out_df = out_df.rename(columns = {'File Name':'ms_file'})
out_df.head(2)

,ms_file,Position,plate
0,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A2,SA001
1,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A3,SA001


In [18]:
out_df['RUN_DATE'] = out_df.ms_file.str.split('RG').apply(lambda x: x[0].replace('_','-'))
out_df['WELL_ROW'] = out_df.Position.str.split(':').apply(lambda x: x[1][:1])
out_df['WELL_COL'] = out_df.Position.str.split(':').apply(lambda x: x[1][1:])
out_df['METHOD'] = out_df.ms_file.str.split('HILIC').apply(lambda x: x[-1][:3])
out_df['BI_NBR'] = out_df.ms_file.str.split('RPT001_').apply(lambda x: x[-1])
out_df['FILE_SIZE'] = ms_files.FILE_SIZE
out_df = out_df.drop('Position'])
out_df.head(2)

,ms_file,Position,plate,RUN_DATE,WELL_ROW,WELL_COL,METHOD,BI_NBR,FILE_SIZE
0,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A2,SA001,2020-05-11,A,2,Neg,BI_16_0013,22479052
1,2020_05_11RG_HILICNeg15S_Col002_LSARP_SA001_RP...,G:A3,SA001,2020-05-11,A,3,Neg,BI_16_0017,22452139


In [23]:
from datetime import date
# str(date.today()).replace('-','')

In [24]:
out_df.to_csv(str(date.today()).replace('-','')+'_metadata_file.csv', index = False)